`Prerequisites`

To complete this exercise, you must have access to the `little_lemon` database. As an authorized user, you need to establish a connection between Python and the database via the connector API and create a cursor object

Once the connection is established, and you have a cursor object, you can select the database little_lemon and print the names of the tables

You should have the four created tables from previous labs in your database envionrment


#### **Learning objectives:**
* Work with date and time functions in Python

`Scenario:`

Little Lemon needs to schedule its staff’s duties according to the restaurant’s peak hours. They also want to display the guest’s name and their expected arrival time kitchen screen to keep the staff informed. These and several other similar tasks, such as changing the booking time, require working with the date and time column. 

You are tasked to help Little Lemon work with the date and time columns using Python for their Python-based application.

In [1]:
import mysql.connector as connector

try:
    connection=connector.connect(user="root", password="")
    print('Connection established!')
except connector.Error as e:
    print(f"""
    Error code : {e.errno}|
    Error message : {e.msg}
    """)

Connection established!


In [2]:
cursor = connection.cursor()

# select little_lemon db
cursor.execute('USE little_lemon;')

# print out db in use
f'We are connected to : {connection.database}'

'We are connected to : little_lemon'

### **Task 1:**

Little Lemon wants to retrieve the number of bookings in each hour so that they can schedule their staff’s duties accordingly.

`Expected Output:`
![Task 1](images/2_1.png)

In [6]:
upcoming_bookings = """
SELECT
    HOUR(BookingSlot) AS Hour,
    COUNT(*) AS Bookings
FROM Bookings
GROUP BY Hour
ORDER BY Hour
"""
cursor.execute(upcoming_bookings)
print('Upcoming Bookings')
for row in cursor.fetchall():
    print(f'Hour: {row[0]} <<>> {row[1]} Booking/s')

Upcoming Bookings
Hour: 15 <<>> 1 Booking/s
Hour: 17 <<>> 1 Booking/s
Hour: 18 <<>> 1 Booking/s
Hour: 19 <<>> 2 Booking/s
Hour: 20 <<>> 1 Booking/s


### **Task 2:**

Little Lemon needs to display the following information for their staff:
* Each guest’s table number,
* Each guest’s full name,
* And the expected arrival time in hours and minutes of each guest (e.g., 15 hours and 0 mins).

![Datetime Output 2](images/2_2.png)

In [24]:
from datetime import datetime as dt
from datetime import timedelta as td
print(dt.now(), type(dt.now()), type(dt.today()))
print(dt.date(dt.now())) # have to pass a datetime to dt.date
print(dt.time(dt.today())) # needs another datetime
print(dt.now() + td(hours=2)) # add a 'time delta' to a datetime object

2023-07-11 12:45:30.725297 <class 'datetime.datetime'> <class 'datetime.datetime'>
2023-07-11
12:45:30.725956
2023-07-11 14:45:30.726521


In [22]:
print(dir(dt.time(dt.today())))
# https://pythontic.com/datetime/datetime/time

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'dst', 'fold', 'fromisoformat', 'hour', 'isoformat', 'max', 'microsecond', 'min', 'minute', 'replace', 'resolution', 'second', 'strftime', 'tzinfo', 'tzname', 'utcoffset']


In [29]:
(type(dt.time(dt.today()).minute), dt.time(dt.today()).minute), type(dt.time(dt.today()).strftime)

((int, 48), builtin_function_or_method)

In [34]:
try_time = """
SELECT 
    BookingSlot
FROM Bookings
"""
cursor.execute(try_time)
for row in cursor.fetchall():
    #print(row[0], type(row[0]))
    booking = row[0]
    # Change a time delta object (received from query) to datetime strptime (needs a string not timedelta though)
    booking_dt = dt.strptime(str(booking), '%H:%M:%S')
    print(booking, type(booking), booking_dt, type(booking_dt))

19:00:00 <class 'datetime.timedelta'> 1900-01-01 19:00:00 <class 'datetime.datetime'>
19:00:00 <class 'datetime.timedelta'> 1900-01-01 19:00:00 <class 'datetime.datetime'>
15:00:00 <class 'datetime.timedelta'> 1900-01-01 15:00:00 <class 'datetime.datetime'>
17:30:00 <class 'datetime.timedelta'> 1900-01-01 17:30:00 <class 'datetime.datetime'>
18:30:00 <class 'datetime.timedelta'> 1900-01-01 18:30:00 <class 'datetime.datetime'>
20:00:00 <class 'datetime.timedelta'> 1900-01-01 20:00:00 <class 'datetime.datetime'>


* Now let's solve it

In [40]:
guest_booking_slots = """
SELECT 
    TableNo,
    CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
    BookingSlot
FROM Bookings
ORDER BY HOUR(BookingSlot)
"""
# Execute query 
cursor.execute(guest_booking_slots)

# Fetch all results from cursor
results = cursor.fetchall()
# print default output before iteration
print('The guests and their booking slots are:')
for row in results:
    table_no = row[0]
    booking_name = row[1]
    booking_datetime = dt.strptime(str(row[2]), '%H:%M:%S')
    # assign to some values then build string
    booking_dt_hours, booking_dt_mins = booking_datetime.hour, booking_datetime.minute
    print(f'[Table no:] {table_no} >> {booking_name} is expected to arrive at: {booking_dt_hours} hrs and {booking_dt_mins} mins')

The guests and their booking slots are:
[Table no:] 19 >> Vanessa McCarthy is expected to arrive at: 15 hrs and 0 mins
[Table no:] 15 >> Marcos Romero is expected to arrive at: 17 hrs and 30 mins
[Table no:] 5 >> Hiroki Yamane is expected to arrive at: 18 hrs and 30 mins
[Table no:] 12 >> Anna Iversen is expected to arrive at: 19 hrs and 0 mins
[Table no:] 12 >> Joakim Iversen is expected to arrive at: 19 hrs and 0 mins
[Table no:] 8 >> Diana Pinto is expected to arrive at: 20 hrs and 0 mins


### **Task 3:**
* A guest with `booking ID 2` and `table number 12` wants to change their arrival time by one hour from 7pm to 8pm. 

`Expected Output`

![Task 3 Duration Change](images/2_3.png)

In [49]:
booking_change = """
SELECT 
    BookingID,
    TableNo,
    BookingSlot
FROM Bookings
WHERE BookingID = 2
"""

# Execute and fetch
cursor.execute(booking_change)

results = cursor.fetchall()

print("Booking time change ALERT!!")
for row in results:
    booking_datetime = dt.strptime(str(row[2]), '%H:%M:%S')
    new_arrival = booking_datetime + td(hours=1)
    print(
    f"""Booking ID: {row[0]}
Table number: {row[1]}
Booked slot: {booking_datetime.time()} 
New arrival time: {new_arrival.time()}
    """)
# # Can call the time to just pull the time

Booking time change ALERT!!
Booking ID: 2
Table number: 12
Booked slot: 19:00:00 
New arrival time: 20:00:00
    


```python
"""UPDATE Bookings 
SET BookingSlot=ADDTIME(BookingSlot,"1:00:00") 
WHERE BookingID=2;"""

# The SQL query is: 
sql_query = """SELECT 
BookingID, 
TableNo, 
BookingSlot, 
ADDTIME(BookingSlot,"1:00:00") as NewTime 
FROM Bookings
WHERE TableNo = 12 AND BookingID = 2;"""
```

* Could do something like this too with the ADDTIME function

In [50]:
# Let's close the cursor and the connection
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
